In [1]:
import pandas as pd
import seaborn as sns
from sklearn import preprocessing
from sklearn.linear_model import LinearRegression
from sklearn import preprocessing
import lightgbm as lgb
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import accuracy_score
from sklearn import metrics

In [2]:
train_data = pd.read_csv("duom/train_data.csv")
test_data = pd.read_csv("duom/test_data.csv")

########################################
Išrinamos kategorinės reikšmės, kurių nėra testiniame duomenų rinkinyje
########################################

In [3]:
for column in train_data:
    if train_data[column].dtype == "object":
        print(column)
        print(train_data[column].unique())
        print(len(train_data[column].unique()))
        print(set(train_data[column].unique()).difference(set(test_data[column].unique())))
        print(set(test_data[column].unique()).difference(set(train_data[column].unique())))
        print("")

term
['short' 'long']
2
set()
set()

credit_score
['very_good' 'good' nan 'fair']
4
set()
set()

loan_purpose
['other' 'debt_consolidation' 'home_improvements' 'take_a_trip'
 'medical_bills' 'educational_expenses' 'major_purchase' 'small_business'
 'business_loan' 'buy_a_car' 'moving' 'buy_house' 'vacation' 'wedding'
 'renewable_energy' 'UFVCU1VC']
16
{'renewable_energy', 'UFVCU1VC'}
set()

home_ownership
['rent' 'mortgage' 'own' nan]
4
{nan}
set()



In [4]:
train_data.drop(train_data[train_data['loan_purpose'] == 'renewable_energy'].index, inplace=True)
train_data.drop(train_data[train_data['loan_purpose'] == 'UFVCU1VC'].index, inplace=True)

########################################
Trūkstamų reikšmių tikrinimas
########################################

In [5]:
train_data.isna().sum()

id                                    0
y                                     0
amount_current_loan                   0
term                                  0
credit_score                    2096226
loan_purpose                          0
yearly_income                   1687689
home_ownership                        1
bankruptcies                      16195
years_current_job                371020
monthly_debt                          1
years_credit_history                  1
months_since_last_delinquent    4665726
open_accounts                         1
credit_problems                       1
credit_balance                        1
max_open_credit                     175
dtype: int64

########################################
Ištrinamos arba užpildomos trūkstamos reikšmės
########################################

In [6]:
train_data.drop(train_data[(train_data['credit_score'].isna()) 
     & (train_data['months_since_last_delinquent'].isna()) 
     & (train_data['yearly_income'].isna())].index, inplace=True)

train_data.drop(train_data[train_data["max_open_credit"].isnull()].index, inplace = True) 
train_data.drop(train_data[train_data["bankruptcies"].isnull()].index, inplace = True) 
train_data.drop(train_data[train_data["years_current_job"].isnull()].index, inplace = True) 
#train_data.drop(train_data[train_data["credit_score"].isnull()].index, inplace = True) 

train_data["months_since_last_delinquent"] = train_data["months_since_last_delinquent"].fillna(value = 200)
test_data["months_since_last_delinquent"] = test_data["months_since_last_delinquent"].fillna(value = 200)

In [7]:
train_data.isna().sum()

id                                    0
y                                     0
amount_current_loan                   0
term                                  0
credit_score                    1113307
loan_purpose                          0
yearly_income                    726402
home_ownership                        0
bankruptcies                          0
years_current_job                     0
monthly_debt                          0
years_credit_history                  0
months_since_last_delinquent          0
open_accounts                         0
credit_problems                       0
credit_balance                        0
max_open_credit                       0
dtype: int64

In [8]:
train_data

,id,y,amount_current_loan,term,credit_score,loan_purpose,yearly_income,home_ownership,bankruptcies,years_current_job,monthly_debt,years_credit_history,months_since_last_delinquent,open_accounts,credit_problems,credit_balance,max_open_credit
0,1,0,386342,short,very_good,other,1371971.0,rent,0.0,4.0,5281.62,17.0,25.0,9.0,0.0,59356.0,388828.0
1,2,0,429000,short,good,debt_consolidation,823042.0,mortgage,0.0,10.0,13854.61,19.0,10.0,15.0,0.0,344584.0,811778.0
4,5,0,654126,short,good,debt_consolidation,1242847.0,rent,0.0,0.0,27490.15,28.0,68.0,8.0,1.0,265259.0,456214.0
5,6,0,133034,short,good,debt_consolidation,651035.0,rent,0.0,10.0,6473.11,30.0,200.0,12.0,0.0,102771.0,387662.0
6,7,0,317570,short,good,debt_consolidation,1937601.0,mortgage,0.0,10.0,34154.97,17.0,200.0,12.0,0.0,171684.0,240438.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8774106,8774107,1,176308,short,NaN,buy_a_car,1807888.0,own,1.0,4.0,23347.58,21.0,200.0,15.0,1.0,65512.0,352352.0
8774107,8774108,1,531036,short,NaN,debt_consolidation,NaN,mortgage,0.0,10.0,15426.29,24.0,21.0,10.0,0.0,338485.0,499466.0
8774109,8774110,0,649770,long,NaN,debt_consolidation,NaN,mortgage,0.0,1.0,27345.75,16.0,75.0,8.0,0.0,434378.0,1317404.0
8774110,8774111,0,142010,short,good,debt_consolidation,766688.0,mortgage,0.0,10.0,15732.38,18.0,70.0,6.0,0.0,370899.0,572748.0


In [9]:
train_data.isna().sum()

id                                    0
y                                     0
amount_current_loan                   0
term                                  0
credit_score                    1113307
loan_purpose                          0
yearly_income                    726402
home_ownership                        0
bankruptcies                          0
years_current_job                     0
monthly_debt                          0
years_credit_history                  0
months_since_last_delinquent          0
open_accounts                         0
credit_problems                       0
credit_balance                        0
max_open_credit                       0
dtype: int64

########################################
Užpildomos trūkstamos yearly_income reikšmės
########################################

In [10]:
train_data_inc_isnull = train_data[train_data['yearly_income'].isnull()]
train_data_inc_notnull = train_data[train_data['yearly_income'].notnull()]

train_data.dropna(inplace=True)
x_train = train_data_inc_notnull[['monthly_debt','credit_balance','max_open_credit','y']]
y_train = train_data_inc_notnull['yearly_income']

lr = LinearRegression()
lr.fit(x_train, y_train)

x_test = train_data_inc_isnull[['monthly_debt','credit_balance','max_open_credit','y']]
y_pred = lr.predict(x_test).astype(int)

train_data_inc_isnull['yearly_income'] = y_pred

train_data = pd.concat([train_data_inc_isnull, train_data_inc_notnull])

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [11]:
test_data_inc_isnull = test_data[test_data['yearly_income'].isnull()]
test_data_inc_notnull = test_data[test_data['yearly_income'].notnull()]

x_test = test_data_inc_notnull[['monthly_debt','credit_balance','max_open_credit']]
y_test = test_data_inc_notnull['yearly_income']

lr = LinearRegression()
lr.fit(x_test, y_test)

x_test = test_data_inc_isnull[['monthly_debt','credit_balance','max_open_credit']]
y_pred = lr.predict(x_test).astype(int)

test_data_inc_isnull['yearly_income'] = y_pred

test_data = pd.concat([test_data_inc_isnull, test_data_inc_notnull])

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


########################################
Užpildomos trūkstamos credit_score reikšmės
########################################

In [12]:
train_data_scr_isnull = train_data[train_data['credit_score'].isnull()]
train_data_scr_notnull = train_data[train_data['credit_score'].notnull()]

le = preprocessing.LabelEncoder()
train_data_scr_notnull['credit_score'] = le.fit_transform(train_data_scr_notnull['credit_score'])
print(le.classes_)

x_train = train_data_scr_notnull[['amount_current_loan','monthly_debt','credit_balance','max_open_credit','y']]
y_train = train_data_scr_notnull['credit_score']

lr = LinearRegression()
lr.fit(x_train, y_train)

x_test = train_data_scr_isnull[['amount_current_loan','monthly_debt','credit_balance','max_open_credit','y']]
y_pred = lr.predict(x_test).astype(int)

train_data_scr_isnull['credit_score'] = y_pred

train_data = pd.concat([train_data_scr_isnull, train_data_scr_notnull])
train_data['credit_score'] = le.inverse_transform(train_data['credit_score'])

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


['fair' 'good' 'very_good']


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()


In [13]:
test_data_scr_isnull = test_data[test_data['credit_score'].isnull()]
test_data_scr_notnull = test_data[test_data['credit_score'].notnull()]

le = preprocessing.LabelEncoder()
test_data_scr_notnull['credit_score'] = le.fit_transform(test_data_scr_notnull['credit_score'])
print(le.classes_)

x_test = test_data_scr_notnull[['amount_current_loan','monthly_debt','credit_balance','max_open_credit']]
y_test = test_data_scr_notnull['credit_score']

lr = LinearRegression()
lr.fit(x_test, y_test)

x_test = test_data_scr_isnull[['amount_current_loan','monthly_debt','credit_balance','max_open_credit']]
y_pred = lr.predict(x_test).astype(int)

test_data_scr_isnull['credit_score'] = y_pred

test_data = pd.concat([test_data_scr_isnull, test_data_scr_notnull])
test_data['credit_score'] = le.inverse_transform(test_data['credit_score'])

['fair' 'good' 'very_good']


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()


In [14]:
train_data.describe().applymap(lambda x: f"{x:0.3f}")

,id,y,amount_current_loan,yearly_income,bankruptcies,years_current_job,monthly_debt,years_credit_history,months_since_last_delinquent,open_accounts,credit_problems,credit_balance,max_open_credit
count,7500301.000,7500301.000,7500301.000,7500301.000,7500301.000,7500301.000,7500301.000,7500301.000,7500301.000,7500301.000,7500301.000,7500301.000,7500301.000
mean,4396638.979,0.239,12989713.386,1397737.193,0.114,5.921,18780.363,18.103,113.786,11.235,0.171,297659.807,763285.454
std,2528804.127,0.426,33207047.660,1102706.819,0.347,3.629,12219.079,6.819,83.955,5.052,0.492,378833.106,7604442.200
min,1.000,0.000,10802.000,76627.000,0.000,0.000,0.000,4.000,0.000,0.000,0.000,0.000,0.000
25%,2209923.000,0.000,187616.000,880897.000,0.000,3.000,10514.980,14.000,30.000,8.000,0.000,113240.000,275902.000
50%,4398359.000,0.000,323378.000,1214670.000,0.000,6.000,16527.530,17.000,76.000,10.000,0.000,210596.000,470888.000
75%,6586163.000,0.000,539506.000,1668447.000,0.000,10.000,24325.890,22.000,200.000,14.000,0.000,369987.000,788722.000
max,8774112.000,1.000,99999999.000,165557393.000,7.000,10.000,435843.280,70.000,200.000,76.000,15.000,32878968.000,1539737892.000


In [15]:
train_data.describe().applymap(lambda x: f"{x:0.3f}")

,id,y,amount_current_loan,yearly_income,bankruptcies,years_current_job,monthly_debt,years_credit_history,months_since_last_delinquent,open_accounts,credit_problems,credit_balance,max_open_credit
count,7500301.000,7500301.000,7500301.000,7500301.000,7500301.000,7500301.000,7500301.000,7500301.000,7500301.000,7500301.000,7500301.000,7500301.000,7500301.000
mean,4396638.979,0.239,12989713.386,1397737.193,0.114,5.921,18780.363,18.103,113.786,11.235,0.171,297659.807,763285.454
std,2528804.127,0.426,33207047.660,1102706.819,0.347,3.629,12219.079,6.819,83.955,5.052,0.492,378833.106,7604442.200
min,1.000,0.000,10802.000,76627.000,0.000,0.000,0.000,4.000,0.000,0.000,0.000,0.000,0.000
25%,2209923.000,0.000,187616.000,880897.000,0.000,3.000,10514.980,14.000,30.000,8.000,0.000,113240.000,275902.000
50%,4398359.000,0.000,323378.000,1214670.000,0.000,6.000,16527.530,17.000,76.000,10.000,0.000,210596.000,470888.000
75%,6586163.000,0.000,539506.000,1668447.000,0.000,10.000,24325.890,22.000,200.000,14.000,0.000,369987.000,788722.000
max,8774112.000,1.000,99999999.000,165557393.000,7.000,10.000,435843.280,70.000,200.000,76.000,15.000,32878968.000,1539737892.000


In [16]:
train_data = train_data.sort_values('id')
test_data = test_data.sort_values('id')
train_data.to_csv("duom/train_data_2.csv", sep=',', encoding='utf-8', index=False)
test_data.to_csv("duom/test_data_2.csv", sep=',', encoding='utf-8', index=False)